In [36]:
import pandas as pd
import numpy as np
path_sub = "./sub/"
path_build = "./build/"
from utils import *
users = pd.read_pickle(f"{path_build}/train_user.pkl")
users.sort_values(by=['user_id'], ascending=[True], inplace=True)
users = users.reset_index(drop=True)

from sklearn.model_selection import KFold

def fold5_mean(filename):
    gender_preds = []
    age_preds = []
    kfolder = KFold(n_splits=5, shuffle=True, random_state=2020)
    kfold = kfolder.split(users[0:3000000])
    i = 1
    gender_oof = np.zeros((3000000, 2))
    age_oof = np.zeros((3000000, 10))

    for train_index, vali_index in kfold:
        print(i,train_index, vali_index)
        age_prob = np.load(f"{path_sub}/{filename}/{i}/age_prob.npy")
        gender_prob = np.load(f"{path_sub}/{filename}/{i}/gender_prob.npy")
        val_age_prob = np.load(f"{path_sub}/{filename}/{i}/val_age_prob.npy")
        val_gender_prob = np.load(f"{path_sub}/{filename}/{i}/val_gender_prob.npy")
        
        gender_preds.append(gender_prob)
        age_preds.append(age_prob)
        gender_oof[vali_index] = val_gender_prob
        age_oof[vali_index] = val_age_prob
        
        
        i = i+1
    gender_prob = np.array(gender_preds).mean(axis=0)
    age_prob = np.array(age_preds).mean(axis=0)
    
    age_accuracy = accuracy_score(y_true=users['age'][0:3000000]-1, y_pred=age_oof.argmax(axis=1))
    gender_accuracy = accuracy_score(y_true=users['gender'][0:3000000]-1, y_pred=gender_oof.argmax(axis=1))
    print(f'age accuracy_score:{age_accuracy}')
    print(f'gender accuracy_score:{gender_accuracy}, sum:{age_accuracy+gender_accuracy}')
    print(gender_prob.sum(), age_prob.sum())
    print('----------------------------------------------')
    return gender_prob, age_prob, gender_oof,age_oof

    
# 两个五折
gender_prob_51, age_prob_51, gender_oof_51, age_oof_51 = fold5_mean('sub-5-5fold')
gender_prob_52, age_prob_52, gender_oof_52, age_oof_52 = fold5_mean('sub-5times-5fold')
gender_prob_53, age_prob_53, gender_oof_53, age_oof_53 = fold5_mean('sub-10-5fold')


# LR
gender_prob_lr = np.load(f"{path_sub}/lr/gender_prob.npy")
age_prob_lr = np.load(f"{path_sub}/lr/age_prob.npy")
gender_val_prob_lr = np.load(f"{path_sub}/lr/val_gender_prob.npy")
age_val_prob_lr = np.load(f"{path_sub}/lr/val_age_prob.npy")


train = pd.DataFrame()
test = pd.DataFrame()

for i in range(10):
    train[f'age1_{i}'] =  age_oof_51[:,i]
    train[f'age2_{i}'] =  age_oof_52[:,i]
    train[f'age3_{i}'] =  age_oof_53[:,i]
    train[f'age4_{i}'] =  age_val_prob_lr[:,i]
    
    test[f'age1_{i}'] =  age_prob_51[:,i]
    test[f'age2_{i}'] =  age_prob_52[:,i]
    test[f'age3_{i}'] =  age_prob_53[:,i]
    test[f'age4_{i}'] =  age_prob_lr[:,i]
    
    
    if i < 2:
        train[f'gender1_{i}'] =  gender_oof_51[:,i]
        train[f'gender2_{i}'] =  gender_oof_52[:,i]
        train[f'gender3_{i}'] =  gender_oof_53[:,i]
        train[f'gender4_{i}'] =  gender_val_prob_lr[:,i]
        
        test[f'gender1_{i}'] =  gender_prob_51[:,i]
        test[f'gender2_{i}'] =  gender_prob_52[:,i]
        test[f'gender3_{i}'] =  gender_prob_53[:,i]
        test[f'gender4_{i}'] =  gender_prob_lr[:,i]

# from sklearn.naive_bayes import MultinomialNB
# clf = MultinomialNB()
# age_pred = clf.predict(test)

from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(solver='saga', multi_class='ovr', n_jobs=-1)
reg_model_age = reg.fit(train, users['age'])
age_train_pred = reg_model_age.predict(train)

reg = LogisticRegression(solver='saga', multi_class='ovr', n_jobs=-1)
reg_model_gender = reg.fit(train, users['gender'])
gender_train_pred = reg_model_gender.predict(train)


age_accuracy1 = accuracy_score(y_true=users['age'][0:3000000], y_pred=age_train_pred)
age_accuracy2 = accuracy_score(y_true=users['gender'][0:3000000], y_pred=gender_train_pred)

age_accuracy1,age_accuracy2,age_accuracy1+age_accuracy2
#predict_proba

1 [      1       6       8 ... 2999997 2999998 2999999] [      0       2       3 ... 2999990 2999993 2999994]
2 [      0       1       2 ... 2999997 2999998 2999999] [      9      24      30 ... 2999984 2999987 2999989]
3 [      0       1       2 ... 2999994 2999995 2999998] [      6       8      13 ... 2999996 2999997 2999999]
4 [      0       1       2 ... 2999997 2999998 2999999] [     10      11      16 ... 2999991 2999992 2999995]
5 [      0       2       3 ... 2999996 2999997 2999999] [      1      12      20 ... 2999976 2999983 2999998]
age accuracy_score:0.49995266666666666
gender accuracy_score:0.9480696666666667, sum:1.4480223333333333
1000000.0 999999.25
----------------------------------------------
1 [      1       6       8 ... 2999997 2999998 2999999] [      0       2       3 ... 2999990 2999993 2999994]
2 [      0       1       2 ... 2999997 2999998 2999999] [      9      24      30 ... 2999984 2999987 2999989]
3 [      0       1       2 ... 2999994 2999995 2999998] [  

/opt/conda/envs/tensorflow2_py3/lib/python3.6/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## 研究模型之间的差异性

In [81]:
import pandas as pd
import numpy as np
path_sub = "./sub/"
path_build = "./build/"
from utils import *
users = pd.read_pickle(f"{path_build}/train_user.pkl")
users.sort_values(by=['user_id'], ascending=[True], inplace=True)
users = users.reset_index(drop=True)

from sklearn.model_selection import KFold

def fold5_mean(filename):
    gender_preds = []
    age_preds = []
    kfolder = KFold(n_splits=5, shuffle=True, random_state=2020)
    kfold = kfolder.split(users[0:3000000])
    i = 1
    gender_oof = np.zeros((3000000, 2))
    age_oof = np.zeros((3000000, 10))

    for train_index, vali_index in kfold:
        print(i,train_index, vali_index)
        age_prob = np.load(f"{path_sub}/{filename}/{i}/age_prob.npy")
        val_age_prob = np.load(f"{path_sub}/{filename}/{i}/val_age_prob.npy")
        try:
            gender_prob = np.load(f"{path_sub}/{filename}/{i}/gender_prob.npy")
            val_gender_prob = np.load(f"{path_sub}/{filename}/{i}/val_gender_prob.npy")
        except:
            print('gender not exists!!!')
            gender_prob = 1
            val_gender_prob = 1
        gender_preds.append(gender_prob)
        age_preds.append(age_prob)
        gender_oof[vali_index] = val_gender_prob
        age_oof[vali_index] = val_age_prob
        
        
        i = i+1
    gender_prob = np.array(gender_preds).mean(axis=0)
    age_prob = np.array(age_preds).mean(axis=0)
    
    age_accuracy = accuracy_score(y_true=users['age'][0:3000000]-1, y_pred=age_oof.argmax(axis=1))
    gender_accuracy = accuracy_score(y_true=users['gender'][0:3000000]-1, y_pred=gender_oof.argmax(axis=1))
    print(f'age accuracy_score:{age_accuracy}')
    print(f'gender accuracy_score:{gender_accuracy}, sum:{age_accuracy+gender_accuracy}')
    print(gender_prob.sum(), age_prob.sum())
    print('----------------------------------------------')
    return gender_prob, age_prob, gender_oof,age_oof

    
# 两个五折
gender_prob_51, age_prob_51, gender_oof_51, age_oof_51 = fold5_mean('sub-5-5fold')
gender_prob_52, age_prob_52, gender_oof_52, age_oof_52 = fold5_mean('sub-5times-5fold')
gender_prob_53, age_prob_53, gender_oof_53, age_oof_53 = fold5_mean('sub-10-5fold')
gender_prob_54, age_prob_54, gender_oof_54, age_oof_54 = fold5_mean('sub-5unit*1-5fold')
gender_prob_55, age_prob_55, gender_oof_55, age_oof_55 = fold5_mean('sub-5timesunit*1-5fold')
gender_prob_56, age_prob_56, gender_oof_56, age_oof_56 = fold5_mean('sub-8-5fold')


# LR
gender_prob_lr = np.load(f"{path_sub}/lr/gender_prob.npy")
age_prob_lr = np.load(f"{path_sub}/lr/age_prob.npy")
gender_val_prob_lr = np.load(f"{path_sub}/lr/val_gender_prob.npy")
age_val_prob_lr = np.load(f"{path_sub}/lr/val_age_prob.npy")

# 单模

age_prob1 = np.load(f"{path_sub}/sub-all_single_model/age_prob.npy")
gender_prob1 = np.load(f"{path_sub}/sub-all_single_model/gender_prob.npy")
val_age_prob1 = np.load(f"{path_sub}/sub-all_single_model/val_age_prob.npy")
val_gender_prob1 = np.load(f"{path_sub}/sub-all_single_model/val_gender_prob.npy")



sub_gender_prob = (gender_prob_51 + gender_prob_52 + gender_prob_53 + gender_prob_54 + gender_prob1 + gender_prob_lr)/6
sub_age_prob = (age_prob_51 + age_prob_52 + age_prob_53 + age_prob_54 + age_prob1 + age_prob_lr)/6
sub_gender = np.argmax(sub_gender_prob ,axis=1) + 1
sub_age = np.argmax(sub_age_prob,axis=1) + 1

val_gender = np.argmax( gender_oof_53 + gender_oof_54 + gender_val_prob_lr ,axis=1) + 1
val_age = np.argmax( age_oof_53 + age_oof_54 + age_oof_56 + age_val_prob_lr,axis=1) + 1
print('*'*50 + '5fold' + '*'*50)
print('age accuracy_score',accuracy_score(y_true=users['age'][0:3000000], y_pred=val_age))
print('gender accuracy_score',accuracy_score(y_true=users['gender'][0:3000000], y_pred=val_gender))
print('age+gender accuracy_score',accuracy_score(y_true=users['gender'][0:3000000], y_pred=val_gender) + accuracy_score(y_true=users['age'][0:3000000], y_pred=val_age))
print('*'*50 + '5fold' + '*'*50)

val_gender1 = np.argmax(gender_oof_51[2700000:3000000] + gender_oof_52[2700000:3000000]  + gender_oof_53[2700000:3000000] + gender_oof_54[2700000:3000000] + val_gender_prob1 + gender_val_prob_lr[2700000:3000000] ,axis=1) + 1
val_age1 = np.argmax(age_oof_51[2700000:3000000] + age_oof_52[2700000:3000000] + age_oof_53[2700000:3000000] + age_oof_54[2700000:3000000] + val_age_prob1 + age_val_prob_lr[2700000:3000000],axis=1) + 1
print('*'*50 + '5fold+单模' + '*'*50)
print('age accuracy_score',accuracy_score(y_true=users['age'][2700000:3000000], y_pred=val_age1))
print('gender accuracy_score',accuracy_score(y_true=users['gender'][2700000:3000000], y_pred=val_gender1))
print('age+gender accuracy_score',accuracy_score(y_true=users['gender'][2700000:3000000], y_pred=val_gender1) + accuracy_score(y_true=users['age'][2700000:3000000], y_pred=val_age1))
print('*'*50 + '5fold+单模' + '*'*50)

# sub_gender = np.argmax((gender_prob_51 + 
#                        gender_prob_52 + 
#                        gender_prob1 + 
#                        gender_prob2)/4 ,axis=1) + 1

# sub_age = np.argmax((age_prob_51 + 
#                        age_prob_52 + 
#                        age_prob1 + 
#                        age_prob2 )/4 ,axis=1) + 1

print(sub_gender.shape, sub_age.shape)

sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = sub_age
sub['predicted_gender'] = sub_gender
print('ok！')
sub.to_csv(f"{path_sub}/submission.csv", index=False, encoding='utf-8')
np.save(f"{path_sub}/sub_gender_prob", sub_gender_prob)
np.save(f"{path_sub}/sub_age_prob", sub_age_prob)
print(sub_gender_prob.sum(), sub_age_prob.sum(), sub_age_prob.shape, sub_gender_prob.shape)
local2cos(f"{path_sub}/submission.csv", "lht-build-1259079995", "sub")
#https://lht-build-1259079995.cos.ap-guangzhou.myqcloud.com/sub/submission.csv

1 [      1       6       8 ... 2999997 2999998 2999999] [      0       2       3 ... 2999990 2999993 2999994]
2 [      0       1       2 ... 2999997 2999998 2999999] [      9      24      30 ... 2999984 2999987 2999989]
3 [      0       1       2 ... 2999994 2999995 2999998] [      6       8      13 ... 2999996 2999997 2999999]
4 [      0       1       2 ... 2999997 2999998 2999999] [     10      11      16 ... 2999991 2999992 2999995]
5 [      0       2       3 ... 2999996 2999997 2999999] [      1      12      20 ... 2999976 2999983 2999998]
age accuracy_score:0.49995266666666666
gender accuracy_score:0.9480696666666667, sum:1.4480223333333333
1000000.0 999999.25
----------------------------------------------
1 [      1       6       8 ... 2999997 2999998 2999999] [      0       2       3 ... 2999990 2999993 2999994]
2 [      0       1       2 ... 2999997 2999998 2999999] [      9      24      30 ... 2999984 2999987 2999989]
3 [      0       1       2 ... 2999994 2999995 2999998] [  

In [80]:
#两两差异性
from itertools import combinations
for i,j in zip(combinations([age_prob_51,age_prob_52,age_prob_53,age_prob_54,age_prob_55,age_prob_56,age_prob1], 2), combinations([1,2,3,4,5,6,7], 2)):
    print(j,sum(~(np.argmax(i[0],axis=1)==np.argmax(i[1],axis=1))))
    print('-----------------------')
   

    
#两两差异性
from itertools import combinations
for i in combinations([gender_prob_51,gender_prob_52,gender_prob_53,gender_prob_54,gender_prob1], 2):
    print(sum(~(np.argmax(i[0],axis=1)==np.argmax(i[1],axis=1))))
    print('-----------------------')
for i in combinations([1,2,3,4,5,6], 2):
    print(i)
    print('-----------------------')     

(1, 2) 135684
-----------------------
(1, 3) 108691
-----------------------
(1, 4) 135779
-----------------------
(1, 5) 150146
-----------------------
(1, 6) 109742
-----------------------
(1, 7) 129968
-----------------------
(2, 3) 114898
-----------------------
(2, 4) 148910
-----------------------
(2, 5) 114737
-----------------------
(2, 6) 141907
-----------------------
(2, 7) 158196
-----------------------
(3, 4) 133736
-----------------------
(3, 5) 128232
-----------------------
(3, 6) 114731
-----------------------
(3, 7) 137834
-----------------------
(4, 5) 147814
-----------------------
(4, 6) 134739
-----------------------
(4, 7) 149410
-----------------------
(5, 6) 142070
-----------------------
(5, 7) 162513
-----------------------
(6, 7) 135577
-----------------------
7793
-----------------------
5748
-----------------------
8342
-----------------------
8016
-----------------------
6115
-----------------------
9127
-----------------------
9903
-----------------------

In [57]:
#两两差异性
main_model = np.argmax(age_prob12,axis=1)
i = 0
for m in [age_prob0, age_prob1, age_prob2, age_prob3, age_prob4, age_prob5, age_prob6, age_prob7, age_prob8,age_prob9,age_prob10,age_prob11]:
    mm = np.argmax(m,axis=1)
    print(i, ':', mm.shape[0]-sum(mm==main_model))
    i = i+1

0 : 249354
1 : 263231
2 : 269986
3 : 262495
4 : 228121
5 : 229423
6 : 198929
7 : 235225
8 : 238365
9 : 208097
10 : 207562
11 : 202642
